In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os

CLOVASPEECH_API_KEY = os.getenv("CLOVASPEECH_API")
CLOVASPEECH_URL = os.getenv("CLOVASPEECH_INVOKE_URL")


In [3]:
import requests
import json

try:
    from pydub import AudioSegment
except ImportError:
    AudioSegment = None


class ClovaSpeechClient:
    # Clova Speech invoke URL
    invoke_url = CLOVASPEECH_URL
    # Clova Speech secret key
    secret = CLOVASPEECH_API_KEY

    def req_url( # 외부 파일 인식 (url)
        self,
        url,
        completion,
        callback=None,
        userdata=None,
        forbiddens=None,
        boostings=None,
        wordAlignment=True,
        fullText=True,
        diarization=True,
        sed=None,
    ):
        request_body = {
            "url": url,
            "language": "ko-KR",
            "completion": completion,
            "callback": callback,
            "userdata": userdata,
            "wordAlignment": wordAlignment,
            "fullText": fullText,
            "forbiddens": forbiddens,
            "boostings": boostings,
            "diarization": diarization,
            "sed": sed,
        }
        headers = {
            "Accept": "application/json;UTF-8",
            "Content-Type": "application/json;UTF-8",
            "X-CLOVASPEECH-API-KEY": self.secret,
        }
        return requests.post(
            headers=headers,
            url=self.invoke_url + "/recognizer/url",
            data=json.dumps(request_body).encode("UTF-8"),
        )

    def req_object_storage( # Naver Cloud Object Storage에 저장된 파일 인식
        self,
        data_key,
        completion,
        callback=None,
        userdata=None,
        forbiddens=None,
        boostings=None,
        wordAlignment=True,
        fullText=True,
        diarization=None,
        sed=None,
    ):
        request_body = {
            "dataKey": data_key,
            "language": "ko-KR",
            "completion": completion,
            "callback": callback,
            "userdata": userdata,
            "wordAlignment": wordAlignment,
            "fullText": fullText,
            "forbiddens": forbiddens,
            "boostings": boostings,
            "diarization": diarization,
            "sed": sed,
        }
        headers = {
            "Accept": "application/json;UTF-8",
            "Content-Type": "application/json;UTF-8",
            "X-CLOVASPEECH-API-KEY": self.secret,
        }
        return requests.post(
            headers=headers,
            url=self.invoke_url + "/recognizer/object-storage",
            data=json.dumps(request_body).encode("UTF-8"),
        )

    def req_upload( # 로컬 파일 직접 업로드
        self,
        file,
        completion,
        callback=None,
        userdata=None,
        forbiddens=None,
        boostings=None,
        wordAlignment=True,
        fullText=True,
        diarization=None,
        sed=None,
    ):
        request_body = {
            "language": "ko-KR", ### 언어
            "completion": completion, ### 응답방식 [동기 / 비동기]
            "callback": callback, # 비동기 방식일 경우 callback, resultToObs 중 하나 필수 입력
            "userdata": userdata, # 사용자 데이터 세부 정보
            "wordAlignment": wordAlignment, # 인식 결과의 음성과 텍스트 정렬 출력 여부
            "fullText": fullText, # 전체 인식 결과 텍스트 출력 기본 true
            "forbiddens": forbiddens,
            # noiseFiltering : 노이즈 필터링 여부 기본값 true
            "boostings": boostings, ### 키워드 부스팅, 음성 인식률을 높일 수 있는 키워드 목록으로 사용
            "diarization": diarization, ### 화자 인식
            "sed": sed,
        }
        headers = {
            "Accept": "application/json;UTF-8",
            "X-CLOVASPEECH-API-KEY": self.secret,
        }
        print(json.dumps(request_body, ensure_ascii=False).encode("UTF-8"))
        files = {
            "media": open(file, "rb"),
            "params": (
                None,
                json.dumps(request_body, ensure_ascii=False).encode("UTF-8"),
                "application/json",
            ),
        }
        response = requests.post(
            headers=headers, url=self.invoke_url + "/recognizer/upload", files=files
        )
        return response

## 오디오 분할 코드

In [ ]:
# 음성 파일 경로
AUDIO_FILE_PATH = "./data/수집/오디오/team_noise_4p_4m.wav"
# 화자 분리 on off
DIARIZATION = True
BOOSTINGWORDS = None
# BOOSTINGWORDS = ["모태펀드", "대통령"]

In [ ]:
if __name__ == "__main__":
    # --- 1. API 요청 및 응답 받기 ---
    print("클로바 스피치 API를 호출하여 음성 파일을 텍스트로 변환 중...")

    # 오디오 파일의 기본 이름 (확장자 제외)을 추출합니다.
    audio_basename = os.path.splitext(os.path.basename(AUDIO_FILE_PATH))[0] + "_" + str(DIARIZATION)
    
    # 결과를 저장할 새로운 디렉토리 경로를 설정합니다. (./result/오디오파일명/)
    OUTPUT_DIR = os.path.join("./result", audio_basename)
    
    # 출력 디렉토리가 없으면 생성합니다.
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)
        print(f"출력 디렉토리를 생성했습니다: {OUTPUT_DIR}")

    # 출력 파일 경로들을 설정합니다.
    TXT_OUTPUT_PATH = os.path.join(OUTPUT_DIR, f"{audio_basename}.txt")
    JSON_OUTPUT_PATH = os.path.join(OUTPUT_DIR, f"{audio_basename}_result.json")
    
    print(f"변환 결과를 다음 위치에 저장합니다: {OUTPUT_DIR}")

    # 파일이 존재하는지 확인
    if not os.path.exists(AUDIO_FILE_PATH):
        print(f"❌ 오류: 음성 파일 경로를 찾을 수 없습니다: {AUDIO_FILE_PATH}")
    else:
        try:
            # --- API 호출 ---
            # res = requests.Response
            res = ClovaSpeechClient().req_upload(
                file=AUDIO_FILE_PATH, 
                completion="sync",
                # boostings=[
                # {
                #     "words": BOOSTINGWORDS
                # }
                # ]
                diarization={
                    "enable": DIARIZATION, # 화자 분리 코드
                }
            )
            result = res.json()

            if res.status_code != 200:
                print(f"❌ API 호출 실패. 상태 코드: {res.status_code}")
                print(f"응답 본문: {result}")
            else:
                print("✅ 텍스트 변환 성공.")

                # --- 2. 오디오 파일 로드 (분할용) ---
                try:
                    from pydub import AudioSegment
                    audio = AudioSegment.from_wav(AUDIO_FILE_PATH)
                    print("✅ 오디오 파일 로드 성공. 화자별 분할을 시작합니다.")
                except Exception as e:
                    print(f"❌ 오디오 파일 로드 실패: {e}")
                    audio = None

                # --- 3. 화자별 세그먼트 추출, 저장 및 오디오 분할 ---
                segments = result.get("segments", [])
                output_lines = []

                if not segments and result.get("text"):
                    output_lines.append(f"[전체 텍스트]: {result['text']}\n")
                    print(f"[전체 텍스트]: {result['text']}")

                for i, segment in enumerate(segments):
                    # 안전하게 키 추출 (없을 수 있는 필드 대비)
                    speaker_label = segment.get("speaker", {}).get("label", "Unknown")
                    text = segment.get("text", "")
                    start_ms = segment.get("start")
                    end_ms = segment.get("end")
                    confidence = segment.get("confidence")  # ✅ 신뢰도(없을 수 있음)

                    # 시간 문자열 구성 (있을 때만)
                    time_str = ""
                    if start_ms is not None and end_ms is not None:
                        time_str = f"Start: {start_ms/1000:.2f}s - End: {end_ms/1000:.2f}s"
                    elif start_ms is not None:
                        time_str = f"Start: {start_ms/1000:.2f}s"
                    elif end_ms is not None:
                        time_str = f"End: {end_ms/1000:.2f}s"

                    # 라벨에 표시할 보조 정보 구성 (시간/신뢰도 있는 것만)
                    meta_parts = []
                    if confidence is not None:
                        # 보통 0~1 실수로 들어오므로 0.00 형식으로 표기
                        meta_parts.append(f"Confidence: {confidence:.2f}")
                    if time_str:
                        meta_parts.append(time_str)
                    meta = f" ({' | '.join(meta_parts)})" if meta_parts else ""

                    # TXT 파일 내용 포맷팅 (✅ 신뢰도 포함)
                    formatted_line = f"{i:04d}. Speaker {speaker_label}{meta}: {text}\n"
                    output_lines.append(formatted_line)
                    print(formatted_line.strip())

                    # 오디오 분할 및 저장
                    if audio and (start_ms is not None) and (end_ms is not None):
                        try:
                            segment_audio = audio[start_ms:end_ms]

                            # 파일명 형식 변경:
                            # 기존: 1_000_8130-12050.wav
                            # 변경: 0000_Speaker1_8130-12050.wav
                            segment_filename = f"{i:04d}_Speaker{speaker_label}_{start_ms}-{end_ms}.wav"
                            segment_filepath = os.path.join(OUTPUT_DIR, segment_filename)

                            segment_audio.export(segment_filepath, format="wav")
                        except Exception as e:
                            print(f"❌ 오디오 세그먼트 저장 실패: {e}")


                if audio:
                    print(f"✅ 모든 오디오 세그먼트를 {OUTPUT_DIR}에 저장했습니다.")

                # --- 4. 결과를 TXT 및 JSON 파일로 저장 ---
                if output_lines:
                    with open(TXT_OUTPUT_PATH, "w", encoding="utf-8") as f:
                        f.writelines(output_lines)
                    print(f"✅ 변환된 텍스트를 {TXT_OUTPUT_PATH}에 저장했습니다.")

                with open(JSON_OUTPUT_PATH, "w", encoding="utf-8") as jf:
                    json.dump(result, jf, ensure_ascii=False, indent=2)
                print(f"✅ 전체 JSON 결과를 {JSON_OUTPUT_PATH}에 저장했습니다.")

        except requests.exceptions.RequestException as e:
            print(f"❌ 네트워크 또는 요청 오류: {e}")
        except json.JSONDecodeError:
            print("❌ API 응답을 JSON으로 디코딩하는 데 실패했습니다.")
        except Exception as e:
            print(f"❌ 예상치 못한 오류: {e}")

클로바 스피치 API를 호출하여 음성 파일을 텍스트로 변환 중...
출력 디렉토리를 생성했습니다: ./result\science_clean_multilateral_1_5p_3m_False
변환 결과를 다음 위치에 저장합니다: ./result\science_clean_multilateral_1_5p_3m_False
b'{"language": "ko-KR", "completion": "sync", "callback": null, "userdata": null, "wordAlignment": true, "fullText": true, "forbiddens": null, "boostings": null, "diarization": {"enable": false}, "sed": null}'
✅ 텍스트 변환 성공.
✅ 오디오 파일 로드 성공. 화자별 분할을 시작합니다.
0000. Speaker  (Confidence: 0.95 | Start: 0.00s - End: 13.30s): 지구 역사상 가장 지루했던 10억 년 1억 년 사이에 추워졌다 뜨거워졌다 하는 게 지구인데 심지어는 한 6억 년 전에는 적들까지 다 얼어버렸어요. 그래서 한 1 2억 년 단위로 이렇게 출렁출렁거렸는데 약 18억 년
0001. Speaker  (Confidence: 0.98 | Start: 13.30s - End: 25.90s): 부터 8억 년 사이에는 추운 시기가 전혀 없었다. 그다음에 6500만 년 전부터 지금까지 주구장창 온도가 떨어졌어요. 그래서 우리는 지구 역사상 굉장히 추운 시기에 살고 있고요. 2만 년 전에 마지막 빙하기가
0002. Speaker  (Confidence: 0.92 | Start: 25.90s - End: 40.20s): 있었고 상당히 많은 과학자들이 인간이 빙하기를 맞고 있다. 이미 왔었어야 됐다라고 보시는 분들이 많더라고요. 흥미로운 과학 이야기 더욱 재미있게 전해드립니다. 과학을 보다 정용진입니다. 세종대학교에서 은하를 연구하는 우주
0003. Speaker  

## 오디오 분할 없는 코드

In [6]:
# 음성 파일 경로
AUDIO_FILE_PATH = "../data/생성/오디오/1_1.conversation.wav"
# 화자 분리 on off
DIARIZATION = True
BOOSTINGWORDS = None
# BOOSTINGWORDS = ["모태펀드", "대통령"]

In [7]:
if __name__ == "__main__":
    # --- 1. API 요청 및 응답 받기 ---
    print("클로바 스피치 API를 호출하여 음성 파일을 텍스트로 변환 중...")

    # 오디오 파일의 기본 이름 (확장자 제외)을 추출합니다.
    audio_basename = os.path.splitext(os.path.basename(AUDIO_FILE_PATH))[0] # + "_" + str(DIARIZATION)
    
    # 결과를 저장할 새로운 디렉토리 경로를 설정합니다. (../result/오디오파일명/)
    OUTPUT_DIR = os.path.join("../result", audio_basename)
    
    # 출력 디렉토리가 없으면 생성합니다.
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)
        print(f"출력 디렉토리를 생성했습니다: {OUTPUT_DIR}")

    # 출력 파일 경로들을 설정합니다.
    TXT_OUTPUT_PATH = os.path.join(OUTPUT_DIR, f"{audio_basename}.txt")
    JSON_OUTPUT_PATH = os.path.join(OUTPUT_DIR, f"{audio_basename}_result.json")
    
    print(f"변환 결과를 다음 위치에 저장합니다: {OUTPUT_DIR}")

    # 파일이 존재하는지 확인
    if not os.path.exists(AUDIO_FILE_PATH):
        print(f"❌ 오류: 음성 파일 경로를 찾을 수 없습니다: {AUDIO_FILE_PATH}")
    else:
        try:
            # --- API 호출 ---
            res = ClovaSpeechClient().req_upload(
                file=AUDIO_FILE_PATH, 
                completion="sync",
                diarization={
                    "enable": DIARIZATION,  # 화자 분리 코드
                }
            )
            result = res.json()

            if res.status_code != 200:
                print(f"❌ API 호출 실패. 상태 코드: {res.status_code}")
                print(f"응답 본문: {result}")
            else:
                print("✅ 텍스트 변환 성공.")

                # --- 2. 화자별 세그먼트 추출, 저장 및 오디오 분할 ---
                segments = result.get("segments", [])
                output_lines = []

                if not segments and result.get("text"):
                    output_lines.append(f"[전체 텍스트]: {result['text']}\n")
                    print(f"[전체 텍스트]: {result['text']}")

                # --- 3. 세그먼트 정보 처리 (화자별 텍스트 정리) ---
                merged_segments = []
                prev_speaker = None
                prev_start = None
                conf_values = []  # 🔹 화자별 confidence 저장 리스트
                accumulated_text = ""

                for i, segment in enumerate(segments):
                    speaker_label = segment.get("speaker", {}).get("label", "Unknown")
                    text = segment.get("text", "").strip()
                    start_ms = segment.get("start")
                    confidence = segment.get("confidence")

                    # 같은 화자면 텍스트와 confidence를 이어붙임
                    if speaker_label == prev_speaker:
                        accumulated_text += " " + text
                        if confidence is not None:
                            conf_values.append(confidence)
                    else:
                        # 새 화자가 등장하면 이전 화자 저장
                        if prev_speaker is not None:
                            # 평균 신뢰도 계산
                            avg_conf = sum(conf_values) / len(conf_values) if conf_values else None
                            conf_str = f"{avg_conf:.2f}" if avg_conf is not None else "N/A"
                            start_str = f"{int(prev_start):08d}" if prev_start is not None else "00000000"

                            formatted_line = f"{len(merged_segments):04d}:{start_str}:{conf_str}:speaker{prev_speaker}:{accumulated_text.strip()}\n"
                            merged_segments.append(formatted_line)

                        # 새 화자 초기화
                        prev_speaker = speaker_label
                        prev_start = start_ms
                        accumulated_text = text
                        conf_values = [confidence] if confidence is not None else []

                # 마지막 화자 저장
                if prev_speaker is not None:
                    avg_conf = sum(conf_values) / len(conf_values) if conf_values else None
                    conf_str = f"{avg_conf:.2f}" if avg_conf is not None else "N/A"
                    start_str = f"{int(prev_start):08d}" if prev_start is not None else "00000000"
                    formatted_line = f"{len(merged_segments):04d}:{start_str}:{conf_str}:speaker{prev_speaker}:{accumulated_text.strip()}\n"
                    merged_segments.append(formatted_line)

                # 결과 출력 및 저장
                for line in merged_segments:
                    print(line.strip())

                with open(TXT_OUTPUT_PATH, "w", encoding="utf-8") as f:
                    f.writelines(merged_segments)
                print(f"✅ 병합된 텍스트를 {TXT_OUTPUT_PATH}에 저장했습니다.")
                with open(JSON_OUTPUT_PATH, "w", encoding="utf-8") as jf:
                    json.dump(result, jf, ensure_ascii=False, indent=2)
                print(f"✅ 전체 JSON 결과를 {JSON_OUTPUT_PATH}에 저장했습니다.")


        except requests.exceptions.RequestException as e:
            print(f"❌ 네트워크 또는 요청 오류: {e}")
        except json.JSONDecodeError:
            print("❌ API 응답을 JSON으로 디코딩하는 데 실패했습니다.")
        except Exception as e:
            print(f"❌ 예상치 못한 오류: {e}")


클로바 스피치 API를 호출하여 음성 파일을 텍스트로 변환 중...
출력 디렉토리를 생성했습니다: ../result\1_1.conversation
변환 결과를 다음 위치에 저장합니다: ../result\1_1.conversation
b'{"language": "ko-KR", "completion": "sync", "callback": null, "userdata": null, "wordAlignment": true, "fullText": true, "forbiddens": null, "boostings": null, "diarization": {"enable": true}, "sed": null}'
✅ 텍스트 변환 성공.
0000:00000000:0.96:speaker1:안녕하세요. 모두 오늘부터 4주간 재택근무 전면 확대 운영 방안을 확정할 겁니다. 가장 중요한 첫 단추는 생산성 측정 및 관리 방안입니다. 전면 재택 환경에서 생산성을 어떻게 유지할지가 핵심입니다.
0001:00017754:0.94:speaker2:팀장님 생산성 관리를 위해 접속 시간 모니터링 프로그램을 도입하는 게 가장 확실하지 않을까요? 피씨 사용 시간과 활동량을 측정해서 인사팀에 보고하면 객관적일 것 같습니다. 근무시간에 집중할 수 있도록 강제하는 장치가 필요하다고 봅
0002:00035807:0.94:speaker3:죄사원 원 죄송하지만 그 방법은 과거 부분 재택 시도 때 실패한 방식입니다. 당시 피씨 온 오프 시간을 지표로 썼더니 직원들 사이에서 감시당한다는 불만이 터져 나왔고, 심지어 실제 일은 안 하고 마우스만 움직이는 형식적인 접속만 늘어 효율성이 오히려 떨어졌습니다. 직원 신뢰를 해치는 방법으로는 전면 채택을 성공시킬 수 없습니다. 아
0003:00064375:0.96:speaker4:it 측면에서도 단순 접속 시간 측정은 의미가 없습니다. 오히려 개인 정보 문제 등으로 인해 시스템적 오류만 발생시켰던 경험이 있습니다.
0004:00075548:0.95:speaker5:이 과장